# Text Preprocessing
## สมาชิก
> #### นายณัฐชนน ศรีดวงษ์ | 660710114
> #### นายธนินท์ ตั้งกอบลาภ | 660710714

### dataset from kaggle
> ##### Click here for more info -> [SMS Spam Collection Dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

### สำรวจ dataset

In [1]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding='latin1')
df.head() 

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [2]:
df.isna().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

#### ลบคอลัมน์ Unnamed 2, 3, 4

In [3]:
print(df.columns)
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
print("After drop columns", df.columns)
df.head()

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')
After drop columns Index(['v1', 'v2'], dtype='object')


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### มีทั้งหมด 5572 rows

In [4]:
df.shape[0]

5572

In [5]:
# for i in range (df.shape[0]): #df.shape[0] = 5572
#     print(i, df.loc[i,'v2'])
#     print()

#### สังเกตดูจะพบว่า เจออักขระแปลกๆเช่น
| row แรกที่เจอ | อักขระที่เจอ      | ความหมาย      |
| ----------- | ----------- |----------- |
|  18   | `thatåÕs`       |that's      |
|  21 | `IÛ÷m`     |I'm |
|    90  | `DonÛ÷t`       | Don't|
|   90 | `youÛ÷ll`    | you'll|
|   22  | `So Ì_ pay`      | So u pay|
| 19  | `4txt/Ì¼1.20`    | 4txt/£1.20|
|  5   | `å£1500`      | £1500|
| 51  |`&lt;#&gt;`    | <#> (placeholder สำหรับใส่เลข)|
| 28  |`&amp;`    | & |

### โดยอักขระแปลกๆ พวกนี้เกิดจากเหตุผลหลักๆคือในไฟล์ dataset มี 2 อย่างนี้เกิดขึ้น
1. `Encoding ไม่ตรงกัน (decode ผิด)`  
   ไฟล์ข้อความต้นทางน่าจะเป็น Windows-1252 / cp1252 แต่เราอ่านด้วย latin1 (หรือผ่านการแปลงหลายรอบ) ทำให้ byte เดิมถูกแปลเป็นตัวอักษรผิด
   >เลยเห็น åÕ, Û÷, Ì_, å£ แทน ', ’, £ ฯลฯ
3. `ข้อความถูก escape แบบ HTML`  
   บางตัวถูกเขียนให้ปลอดภัยในรูป entity เช่น
   > `&lt;` → <  
   > `&gt;` → >  
   > `&amp;` → &  
   > เลยเห็น `&lt;#&gt;` และ `&amp;` แทน <#> กับ &

> ### 📝 **Note:**  encoding='latin1' คืออะไร?
> เมื่อคอมพิวเตอร์เก็บข้อมูลที่เป็นตัวอักษร จะมีการเข้ารหัส (encode) ตัวอักษรเหล่านั้นให้อยู่ในรูปแบบของตัวเลข (ไบต์) และเมื่อต้องการอ่านข้อมูลกลับคืนมา ก็จะต้องมีการถอดรหัส (decode) ตัวเลขเหล่านั้นให้กลับมาเป็นตัวอักษรเดิม
> - `encoding:` เป็นพารามิเตอร์ในฟังก์ชัน pd.read_csv() (และฟังก์ชันอื่นๆ ที่เกี่ยวข้องกับการอ่านไฟล์) ที่ใช้เพื่อบอกโปรแกรมว่าไฟล์ที่คุณกำลังอ่านนั้นถูกเข้ารหัสด้วยชุดอักขระแบบไหน
> - `latin1` (หรือ ISO-8859-1): เป็นมาตรฐานการเข้ารหัสชุดอักขระแบบหนึ่งที่นิยมใช้กันในอดีต โดยเฉพาะกับภาษาในกลุ่มยุโรปตะวันตก มันสามารถรองรับอักขระพิเศษบางตัวได้ดีกว่าการเข้ารหัสแบบ ascii แต่ไม่ครอบคลุมอักขระทั่วโลกเหมือน utf-8

> ### 📝 **Note:**  HTML escape คืออะไร?
> การแปลง “อักขระพิเศษ” ในข้อความ ให้เป็นรูปแบบที่ ปลอดภัยต่อการแสดงผลใน HTML
เพื่อไม่ให้เบราว์เซอร์เข้าใจผิดว่าเป็นแท็กหรือโค้ด
> #### 🔎 ทำไมต้อง escape
>ใน HTML อักขระบางตัวมีความหมายพิเศษ เช่น
`<` `>` `&` `"` `'`
ถ้าเอามาแสดงตรง ๆ เบราว์เซอร์จะคิดว่าเป็น แท็ก HTML หรือ โค้ด  `__เลยต้อง “escape” มันก่อน__`
>
| ตัวอักษรจริง | HTML escaped | ความหมาย     |
| ------------ | ------------ | ------------ |
| `<`          | `&lt;`       | less than    |
| `>`          | `&gt;`       | greater than |
| `&`          | `&amp;`      | ampersand    |
| `"`          | `&quot;`     | double quote |
| `'`          | `&#39;`      | single quote |
| `<#>`  | `&lt;#&gt;`      | placeholder |

>A gram usually runs like <#>  
>A gram usually runs like 20  
>👉 `<#>` เป็น placeholder แทนตัวเลขราคา

# ข้อสังเกตุที่ค้นพบเกี่ยวกับ Dataset นี้
1. Encoding ไม่ตรงกัน (decode ผิด)  ให้เขียน function dict จัดการ
2. ข้อความถูก escape แบบ HTML ต้อง unescape ก่อน
3. มีลิงค์เช่น www.dbuk.net(row 12) หรือhttp://wap.xxxmobilemovieclub.com?n=QJKGIGHJJGCBL( row 15) ให้จำจัดทิ้ง/เปลี่ยนเป็น`[URL]`
4. อักขระพิเศษ พวกราคา เช่น å£100,000 (row 12) ทำการแทนที่ด้วย `[MONEY]`
5. มีเบอร์โทรให้เปลี่ยน เช่น 08452810075 เปลี่ยนเป็น `[PHONE]`
6. มีคำย่อ เช่น lol, u, omg, n ให้ขยายเป็นคำเต็มก่อน
7. ตัวอักษรมาทั้งพิมพ์เล็กและพิมพ์ใหญ่ผสมกัน

In [6]:
import re
import string
import nltk

nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/natchanon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/natchanon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/natchanon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Preprocessing Data

## ------------- 1) แก้ mojibake / encoding เพี้ยน ------------- #

In [7]:
REPLACEMENTS = {
    "å£": "£",   # แก้ £ ที่เพี้ยน
    "åÕ": "'",   # that's, don't ฯลฯ
    "åÓ": "",    

    "\x89Û÷": "'",   # I'm, don't, we'll ฯลฯ
    "\x89Û_": " ",   # ขยะ text แปลก ๆ ตัดเป็น space

    "Ì_": "u",   # So u pay, msg u, pick u ฯลฯ
    "ÌÏ": "u",   # u wait, u say, u log off
    "Ì¼": "£",   # Ì¼1.20 → £1.20
}

def fix_encoding(text):
    """
    แก้ปัญหาข้อความ encoding เพี้ยน 
    """
    if not isinstance(text, str):     # ถ้าไม่ใช่ string ให้คืนค่าเดิม (กัน error)
        return text

    for bad, good in REPLACEMENTS.items():     # 2) แทนที่ตัวอักษรเพี้ยนตามที่เรารู้จัก
        text = text.replace(bad, good)

    clean_text = ""     # 3) ลบตัวอักษรแปลก ๆ ที่ยังเหลือ     # เก็บไว้เฉพาะ ASCII ปกติ และเครื่องหมาย £
    for ch in text:
        if ord(ch) < 128 or ch == "£":
            clean_text += ch
    return clean_text

## ------------- 2) ขยายคำย่อ ------------- #

In [8]:
abbrev_dict = {
    "2": "to",
    "2day": "today",
    "2moro": "tomorrow",
    "4": "for",
    "abt": "about",
    "asap": "as soon as possible",
    "b": "be",
    "b4": "before",
    "bc": "because",
    "bcoz": "because",
    "bf": "boyfriend",
    "brb": "be right back",
    "bro": "brother",
    "btw": "by the way",
    "c": "see",
    "cal": "call",
    "call": "call",
    "cos": "because",
    "coz": "because",
    "cuz": "because",
    "cya": "see you",
    "dun": "do not",
    "eod": "end of day",
    "ez": "easy",
    "fone": "phone",
    "fyi": "for your information",
    "gf": "girlfriend",
    "gm": "good morning",
    "gn": "good night",
    "gonna": "going to",
    "gotta": "got to",
    "gr8": "great",
    "haha": "laugh",
    "hehe": "laugh",
    "hr": "hour",
    "hrs": "hours",
    "idc": "i do not care",
    "idk": "i do not know",
    "ikr": "i know right",
    "ily": "i love you",
    "imo": "in my opinion",
    "irl": "in real life",
    "jk": "just kidding",
    "k": "okay",
    "kinda": "kind of",
    "lmao": "laughing my ass off",
    "lol": "laugh out loud",
    "luv": "love",
    "min": "minute",
    "mins": "minutes",
    "missu": "miss you",
    "mob": "mobile",
    "msg": "message",
    "n": "and",
    "nah": "no",
    "nite": "night",
    "np": "no problem",
    "nvm": "never mind",
    "ok": "okay",
    "omg": "oh my god",
    "omw": "on my way",
    "pls": "please",
    "plz": "please",
    "pm": "private message",
    "rn": "right now",
    "rofl": "rolling on the floor laughing",
    "sec": "second",
    "secs": "seconds",
    "smh": "shaking my head",
    "sorta": "sort of",
    "sup": "what is up",
    "tba": "to be announced",
    "tbc": "to be confirmed",
    "tbh": "to be honest",
    "tdy": "today",
    "thx": "thanks",
    "tmr": "tomorrow",
    "tmrw": "tomorrow",
    "tnx": "thanks",
    "txt": "text",
    "ty": "thank you",
    "u": "you",
    "ur": "your",
    "u2": "you too",
    "w": "with",
    "wanna": "want to",
    "wk": "week",
    "wks": "weeks",
    "wtf": "what the fuck",
    "wth": "what the hell",
    "xmas": "christmas",
    "xoxo": "hugs and kisses",
    "yo": "hey",
}


In [9]:
import re

def expand_abbrev(text: str) -> str:
    """
    Expand abbreviations using abbrev_dict
    - case-insensitive
    - keep punctuation
    - handle £900, 12hrs, u., msg,
    - avoid breaking phone numbers / ordinals
    """
    tokens = text.split()
    expanded = []

    for token in tokens:
        # แยก prefix / core / suffix
        m = re.match(r"^([^a-zA-Z£]*)([a-zA-Z£]+)([^a-zA-Z£]*)$", token)
        if not m:
            expanded.append(token)
            continue

        prefix, core, suffix = m.groups()
        key = core.lower()

        # กันเลขล้วน / เลขผสมคำ เช่น 2nd, 12th
        if re.search(r"\d", core) and core.lower() not in abbrev_dict:
            expanded.append(token)
            continue

        if key in abbrev_dict:
            expanded_core = abbrev_dict[key]
            expanded.append(prefix + expanded_core + suffix)
        else:
            expanded.append(token)

    return " ".join(expanded)


## ------------- 3) Normalization (lowercase + แทน pattern สำคัญ) ------------- #

In [10]:
URL_RE   = re.compile(r'(https?://\S+|www\.\S+)', flags=re.IGNORECASE)
MONEY_RE = re.compile(r'£\s?\d+([.,]\d+)?')   # £100, £1500, £1.50 ฯลฯ
PHONE_RE = re.compile(r'\b\d{3,}\b')         # ตัวเลขติดกัน 3 ตัวขึ้นไป (มักเป็นเบอร์/โค้ด)

def normalize_text(text: str) -> str:
    # 1) แก้ encoding ก่อน
    text = fix_encoding(text)
    
    # 2) ขยายคำย่อ
    text = expand_abbrev(text)
    # 3) แปลงเป็นตัวเล็ก
    text = text.lower()

    # 4) แทน pattern สำคัญด้วย token พิเศษ
    text = URL_RE.sub(" URL ", text)
    text = MONEY_RE.sub(" MONEY ", text)
    text = PHONE_RE.sub(" PHONE ", text)

    return text


## ------------- 4) ลบ punctuation + digit ------------- #

In [11]:
def remove_punc_num(text: str):
    text = re.sub(r"\d+", " ", text) # ลบตัวเลข
    text = text.translate(str.maketrans("", "", string.punctuation)) # ลบเครื่องหมาย
    return text

## ------------- 5) Tokenize + (optional) stopwords + (optional) stemming ------------- #

In [12]:
def tokenize(text):
    text = re.sub(r"\s+", " ", text).strip()
    return nltk.word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    # กรองเอาเฉพาะคำที่ไม่ใช่ stopword
    return [w for w in tokens if w not in stop_words]

def apply_lemmatization(tokens):
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return tokens

## ------------- 5) รวม pipeline ------------- #

In [13]:
def preprocess_sms(text: str,
                   remove_sw: bool = True,
                   do_stem: bool = True) -> str:
    # step 1: normalize -> แก้ encoding + ขยายคำ + แปลงเป็นตัวเล็ก + แทนคำพิเศษ(URL, MONEY, PHONE)
    text = normalize_text(text)

    # step 2: remove punctuation + digit
    text = remove_punc_num(text)
        
    # step 3: tokenize + stopwords + stemming
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = apply_lemmatization(tokens)

    return tokens
    
    # รวมกลับเป็น string 
    # return " ".join(tokens)

In [14]:
test = "XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message or click here>> http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL."
test = normalize_text(test)
print("normalize_text")
print(test,end="\n\n")

test = remove_punc_num(test)
print("remove_punc_num")
print(test,end="\n\n")

test = tokenize(test)
print("tokenize")
print(test,end="\n\n")

test = remove_stopwords(test)
print("remove_stopwords")
print(test,end="\n\n")

test = apply_lemmatization(test)
print("apply_lemmatization")
print(test,end="\n\n")

normalize_text
xxxmobilemovieclub: to use your credit, click the wap link in the next text message or click here>>  URL  xxxmobilemovieclub.com?n=qjkgighjjgcbl.

remove_punc_num
xxxmobilemovieclub to use your credit click the wap link in the next text message or click here  URL  xxxmobilemovieclubcomnqjkgighjjgcbl

tokenize
['xxxmobilemovieclub', 'to', 'use', 'your', 'credit', 'click', 'the', 'wap', 'link', 'in', 'the', 'next', 'text', 'message', 'or', 'click', 'here', 'URL', 'xxxmobilemovieclubcomnqjkgighjjgcbl']

remove_stopwords
['xxxmobilemovieclub', 'use', 'credit', 'click', 'wap', 'link', 'next', 'text', 'message', 'click', 'URL', 'xxxmobilemovieclubcomnqjkgighjjgcbl']

apply_lemmatization
['xxxmobilemovieclub', 'use', 'credit', 'click', 'wap', 'link', 'next', 'text', 'message', 'click', 'URL', 'xxxmobilemovieclubcomnqjkgighjjgcbl']



In [15]:
# ใช้กับ DataFrame – สร้างคอลัมน์ใหม่ ไม่ทับของเดิม
df["clean_text"] = df["v2"].apply(preprocess_sms)
# # label ก็แปลงเป็น 0/1 ไว้ด้วย
# df["label"] = df["v1"].map({"ham": 0, "spam": 1})

In [16]:
df

,v1,v2,clean_text
0,ham,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazy, available, bugis, g..."
1,ham,Ok lar... Joking wif u oni...,"[okay, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,"[say, early, hor, see, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...","[dont, think, go, usf, life, around, though]"
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[nd, time, tried, contact, MONEY, pound, prize..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[going, esplanade, fr, home]"
5569,ham,"Pity, * was in mood for that. So...any other s...","[pity, mood, soany, suggestion]"
5570,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, id, interested, b..."


# ขั้นตอนที่ทำ Text Preprocessing กับ dataset นี้ มีลำดับดังนี้
1) ตัด cloumn ที่ไม่ได้ใช้ทิ้ง
2) fix encoding และ  fix escape html
3) ขยายคำย่อ เช่น  btw => by the way
4) แปลงเป็นตัวพิมพ์เล็ก
5) แทน URL ด้วย <'URL'>
6) แทน เบอร์โทรศัพท์ ด้วย <'PHONE'>
7) แทน ตัวเลขที่เกี่ยวกับเงินด้วย <'MONEY'>
8) ลบตัวเลข
9) ลบเครื่องหมาย punctuation
10) ตัด space ที่ซ้ำ
11) แบ่งประโยคเป็นคำ ด้วยการทำ tokenization
12) remove stopwords
13) lemmatization